In [40]:
import hail as hl
hl.init()

Py4JJavaError: An error occurred while calling z:is.hail.HailContext.apply.
: java.net.BindException: Cannot assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.AbstractChannelHandlerConte

In [14]:
import pandas as pd
import numpy as np

In [9]:
trait = pd.read_csv('/vol/bmd/yanyul/UKB/gcta_regulability/pheno_martin_et_al_traits_x_British-test-1.tsv', header = 0, sep = '\t')

In [19]:
samples = list(np.array(trait['eid'].to_list()).astype(str))

In [34]:
f = open('/vol/bmd/yanyul/UKB/gcta_regulability/pred_expr_ctimp_Whole_Blood_x_British-test-1.tsv', 'r')
header = f.readline().strip().split('\t')
f.close()

In [39]:
header[-2]

'5129226'

In [37]:
type_dic

{'5595764': dtype('float64'),
 '5172041': dtype('float64'),
 '1528789': dtype('float64'),
 '1517276': dtype('float64'),
 '2468738': dtype('float64'),
 '4556757': dtype('float64'),
 '1484605': dtype('float64'),
 '4185318': dtype('float64'),
 '2261357': dtype('float64'),
 '1221294': dtype('float64'),
 '5010377': dtype('float64'),
 '2778295': dtype('float64'),
 '5253976': dtype('float64'),
 '4126923': dtype('float64'),
 '5169018': dtype('float64'),
 '4777485': dtype('float64'),
 '5519563': dtype('float64'),
 '4237517': dtype('float64'),
 '4763043': dtype('float64'),
 '5395909': dtype('float64'),
 '3856733': dtype('float64'),
 '3181070': dtype('float64'),
 '2657285': dtype('float64'),
 '4566750': dtype('float64'),
 '4452781': dtype('float64'),
 '5216791': dtype('float64'),
 '5686275': dtype('float64'),
 '2556285': dtype('float64'),
 '1506620': dtype('float64'),
 '2321581': dtype('float64'),
 '1003152': dtype('float64'),
 '5453689': dtype('float64'),
 '2939414': dtype('float64'),
 '4548181'

In [36]:
type_dic = {
    k : hl.tfloat for k in header
}
type_dic['gene'] = hl.tstr
pred_expr = hl.import_table('/vol/bmd/yanyul/UKB/gcta_regulability/pred_expr_ctimp_Whole_Blood_x_British-test-1.tsv', types = type_dic)

Initializing Spark and Hail with default parameters...


Py4JJavaError: An error occurred while calling z:is.hail.HailContext.apply.
: java.net.BindException: Cannot assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.AbstractChannelHandlerConte